In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import cv2
import open3d as o3d
from object_pose_estimation import ObjectPoseEstimation, get_box_point_cloud
from kas_camera_utils.realsense_camera import RealsenseCamera
from kas_camera_utils import stream
from copy import deepcopy
from time import time, sleep
from kas_utils.aruco import detect_aruco
from kas_utils import show
from yolov8 import YOLOv8_wrapper

In [ ]:
# initialization

yolov8 = YOLOv8_wrapper(
    "/home/cds-jetson-host/ultralytics/yolov8n-seg-1class.yaml",
    "/home/cds-jetson-host/ultralytics/runs/segment/ToyBox_10/weights/last.pt",
    min_score=0.7)

camera = RealsenseCamera(enable_image=True, enable_depth=True)

toy_box_point_cloud = get_box_point_cloud([0.07, 0.07, 0.07], points_per_cm=7)
toy_box_pose_estimation = ObjectPoseEstimation(toy_box_point_cloud,
    camera.K, camera.D,
    voxel_size=0.005,
    max_correspondence_distances=[0.04, 0.029, 0.018, 0.007])

In [ ]:
image, depth = camera.read(True, True)
scores, classes_ids, boxes, masks = yolov8.segment(image)
if len(scores) != 1:
    raise RuntimeError(f"Detected {len(scores)} boxes, expected 1.")

toy_box_pose = toy_box_pose_estimation.estimate_pose(depth, masks[0])
if toy_box_pose is None:
    raise RuntimeError(f"Could not estimate pose. Reason: {toy_box_pose_estimation.reason}")

In [ ]:
# visualize point clouds

transformed_box_pc = deepcopy(toy_box_pose_estimation.gt_pc)
transformed_box_pc.transform(toy_box_pose)
transformed_box_pc.paint_uniform_color(np.array([1, 0, 0]))
extracted_box_pc = toy_box_pose_estimation.extracted_pc
extracted_box_pc.paint_uniform_color(np.array([0, 1, 0]))
o3d.visualization.draw_geometries([transformed_box_pc, extracted_box_pc])

In [ ]:
# visualize point clouds down

transformed_box_pc_down = deepcopy(toy_box_pose_estimation.gt_pc_down)
transformed_box_pc_down.transform(toy_box_pose)
transformed_box_pc_down.paint_uniform_color(np.array([1, 0, 0]))
extracted_box_pc_down = toy_box_pose_estimation.extracted_pc_down
extracted_box_pc_down.paint_uniform_color(np.array([0, 1, 0]))
o3d.visualization.draw_geometries([transformed_box_pc_down, extracted_box_pc_down])

In [ ]:
# visualize box frame

force_z_axis_up = False
if force_z_axis_up:
    if toy_box_pose[1, 2] > 0:
        correction = np.eye(4)
        correction[0:3, 0:3], _ = cv2.Rodrigues(np.array([np.pi, 0., 0.]))
        pose = np.matmul(toy_box_pose, correction)
else:
    pose = toy_box_pose

draw = image.copy()
rvec, _ = cv2.Rodrigues(pose[0:3, 0:3])
tvec = toy_box_pose[0:3, 3]
cv2.drawFrameAxes(draw, camera.K, camera.D, rvec, tvec, 0.07)
show(draw)